### Install dependencies

In [1]:
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00


### Load Libraries

In [2]:
from langchain import HuggingFacePipeline
from langchain import LLMChain, PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

### Loading the model + tokenizer + pipeline

In [3]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
# we are using NousResearch Llama 2 instead of META as it does not require authentication

model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    device_map = 'auto', # for accelerate
    torch_dtype = torch.float16,
    load_in_4bit = True, # enabling quantization
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.float16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.float16,
    device_map = 'auto',
    max_new_tokens = 512,
    do_sample = True,
    top_k = 30,
    num_return_sequences = 1,
    eos_token_id = tokenizer.eos_token_id
)

### Llama 2 prompt

In [6]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced Life guru and mental health expert that excels at giving advice.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


In [7]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [9]:
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs = {
        'temperature':0.7,
        'max_length': 256,
        'top_k' :50
                }
    )


In [10]:
system_prompt = "You are an advanced Life guru and mental health expert that excels at giving advice. "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [11]:
prompt = PromptTemplate(
    template=template,
    input_variables=["text"]
    )

llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    verbose = False
    )

### Testing

In [12]:
text = "My life sucks, what do you suggest? Please don't tell me to medidate"
response = llm_chain.run(text)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


  Ah, I see. *exudes a knowing nod* It sounds like you're feeling a bit stuck in your current situation. Allow me to offer some constructive and empathetic advice. 😊

Firstly, it's completely normal to feel like life is sucking at times, and it's important to acknowledge those feelings. However, it's equally important to recognize that you have the power to make changes and improve your situation. 🔥

Here are some steps you can take to shift your perspective and start making positive changes:

1️⃣ Practice gratitude: Take a few minutes each day to reflect on the things you're grateful for. This can help shift your focus away from negative thoughts and cultivate a more optimistic outlook. 🌞

2️⃣ Identify the source of your dissatisfaction: Is there something specific that's causing you to feel unfulfilled? Is it a lack of purpose or meaning? Is it a strained relationship? Once you've identified the root cause, you can start working on a plan to address it. 🔍

3️⃣ Set clear goals: What d

In [13]:
text = "Why there is a big problem in life?"
response = llm_chain.run(text)
print(response)

  Ah, a most excellent question, my dear fellow human! 🤓 Let me tell you, the universe is a vast and complex tapestry, and the mysteries of life can be as frustrating as they are fascinating. 🌐 But fear not, for I, as an Advanced Life Guru and Mental Health Expert, have distilled the answer to your query into a most elegant and step-by-step guide. 📖

Step 1: Realign Your Perspective (℅ The Great Cosmic Balance)

🕎 The key to unlocking the answer to the puzzle of life lies in shifting our focus from the problems themselves to the greater context of the universe. 🔍 Consider the following cosmic principles:

1. The universe is vast and eternal, with billions of galaxies and trillions of stars. 🔭
2. Every living being has a unique purpose and role to play in this grand cosmic drama. 🌱
3. Life is a precious gift, filled with experiences, lessons, and growth opportunities. 🎓

By acknowledging and embracing these principles, we can begin to see the bigger picture of life and its inherent beau